In [1]:
import cv2
from cvzone.PoseModule import PoseDetector
import math
import numpy as np
import time
import pickle
import mediapipe as mp
# Drawing helpers
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
import pandas as pd

import os, sys
sys.path.append(os.path.abspath(".."))
from utils.common import load_model, save_model
import warnings
warnings.filterwarnings('ignore')

In [2]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)

In [3]:
IMPORTANT_LMS = [
    "NOSE",
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_HIP",
    "RIGHT_HIP",
    "LEFT_KNEE",
    "RIGHT_KNEE",
    "LEFT_ANKLE",
    "RIGHT_ANKLE"
]

# Tạo các cột cho dữ liệu đầu vào
HEADERS = ["label"]
for landmark in IMPORTANT_LMS:
    for dim in ['x', 'y', 'z']:
        HEADERS.append(f"{landmark.lower()}_{dim}")
        
# IMPORTANT_LMS = [element.lower() for element in IMPORTANT_LMS]

def extract_amd_recalculate_landmarks(pose_landmarks):
    """
    Tịnh tiến thân người vào giữa bức hình, đồng thời dời lại trục toạ độ
    """
    hip_center_x = float((pose_landmarks[23].x + pose_landmarks[24].x) / 2)
    hip_center_y = float((pose_landmarks[23].y + pose_landmarks[24].y) / 2)

    new_center = (0.5, 0.5)
    delta_x = new_center[0] - hip_center_x
    delta_y = new_center[1] - hip_center_y

    data = []
    for landmark in IMPORTANT_LMS:
        # Lấy ra id của key point trên cơ thể người
        key_point_id = mp_pose.PoseLandmark[landmark].value

        key_point = pose_landmarks[key_point_id]
        key_point.x += delta_x - 0.5
        key_point.y += delta_y -0.5
        data.append([key_point.x, key_point.y, key_point.z])

    return np.array(data).flatten().tolist()

### Analyze and detection bad pose

In [4]:
def calculate_distance(pointX, pointY) -> float:
    '''
    Calculate a distance between 2 points
    '''

    x1, y1 = pointX
    x2, y2 = pointY

    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


def analyze_foot_knee_placement(results, stage: str, foot_shoulder_ratio_thresholds: list, knee_foot_ratio_thresholds: dict, visibility_threshold: int) -> dict:
    '''
    Calculate the ratio between the foot and shoulder for FOOT PLACEMENT analysis
    
    Calculate the ratio between the knee and foot for KNEE PLACEMENT analysis

    Return result explanation:
        -1: Unknown result due to poor visibility
        0: Correct knee placement
        1: Placement too tight
        2: Placement too wide
    '''
    analyzed_results = {
        "foot_placement": -1,
        "knee_placement": -1,
    }

    landmarks = results.pose_landmarks.landmark

    # * Visibility check of important landmarks for foot placement analysis
    left_foot_index_vis = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].visibility
    right_foot_index_vis = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].visibility

    left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
    right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility

    # If visibility of any keypoints is low cancel the analysis
    if (left_foot_index_vis < visibility_threshold or right_foot_index_vis < visibility_threshold or left_knee_vis < visibility_threshold or right_knee_vis < visibility_threshold):
        return analyzed_results
    
    # * Calculate shoulder width
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    shoulder_width = calculate_distance(left_shoulder, right_shoulder)

    # * Calculate 2-foot width
    left_foot_index = [landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y]
    right_foot_index = [landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y]
    foot_width = calculate_distance(left_foot_index, right_foot_index)

    # * Calculate foot and shoulder ratio
    foot_shoulder_ratio = round(foot_width / shoulder_width, 1)

    # * Analyze FOOT PLACEMENT
    min_ratio_foot_shoulder, max_ratio_foot_shoulder = foot_shoulder_ratio_thresholds
    if min_ratio_foot_shoulder <= foot_shoulder_ratio <= max_ratio_foot_shoulder:
        analyzed_results["foot_placement"] = 0
    elif foot_shoulder_ratio < min_ratio_foot_shoulder:
        analyzed_results["foot_placement"] = 1
    elif foot_shoulder_ratio > max_ratio_foot_shoulder:
        analyzed_results["foot_placement"] = 2
    
    # * Visibility check of important landmarks for knee placement analysis
    left_knee_vis = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].visibility
    right_knee_vis = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].visibility

    # If visibility of any keypoints is low cancel the analysis
    if (left_knee_vis < visibility_threshold or right_knee_vis < visibility_threshold):
        print("Cannot see foot")
        return analyzed_results

    # * Calculate 2 knee width
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x, landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    knee_width = calculate_distance(left_knee, right_knee)

    # * Calculate foot and shoulder ratio
    knee_foot_ratio = round(knee_width / foot_width, 1)

    # * Analyze KNEE placement
    up_min_ratio_knee_foot, up_max_ratio_knee_foot = knee_foot_ratio_thresholds.get("up")
    middle_min_ratio_knee_foot, middle_max_ratio_knee_foot = knee_foot_ratio_thresholds.get("middle")
    down_min_ratio_knee_foot, down_max_ratio_knee_foot = knee_foot_ratio_thresholds.get("down")

    if stage == "U":
        if up_min_ratio_knee_foot <= knee_foot_ratio <= up_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 0
        elif knee_foot_ratio < up_min_ratio_knee_foot:
            analyzed_results["knee_placement"] = 1
        elif knee_foot_ratio > up_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 2
    elif stage == "M":
        if middle_min_ratio_knee_foot <= knee_foot_ratio <= middle_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 0
        elif knee_foot_ratio < middle_min_ratio_knee_foot:
            analyzed_results["knee_placement"] = 1
        elif knee_foot_ratio > middle_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 2
    elif stage == "D":
        if down_min_ratio_knee_foot <= knee_foot_ratio <= down_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 0
        elif knee_foot_ratio < down_min_ratio_knee_foot:
            analyzed_results["knee_placement"] = 1
        elif knee_foot_ratio > down_max_ratio_knee_foot:
            analyzed_results["knee_placement"] = 2
    
    return analyzed_results



In [5]:
def get_class(encode_label: float):
    return {
        0: "D",
        1: "M",
        2: "U",
    }.get(encode_label, "Unknown")

In [6]:
from utils.common import load_model

model = load_model("./best_models/SVC_model_front.pkl")
input_scaler = load_model("./best_models/input_scaler.pkl")

## Ver original

In [7]:
video_path = 'squat_demo.mp4'
cap = cv2.VideoCapture(video_path)
detector = PoseDetector(detectionCon=0.7, trackCon=0.7)

# Creating Angle finder class
class angleFinder:
    def __init__(self, lmlist, p1, p2, p3, p4, p5, p6, p7, p8, drawPoints):
        self.lmlist = lmlist
        self.p1 = p1
        self.p2 = p2
        self.p3 = p3
        self.p4 = p4
        self.p5 = p5
        self.p6 = p6
        self.p7 = p7
        self.p8 = p8
        self.drawPoints = drawPoints

    # finding angles
    def angle(self):
        if len(self.lmlist) != 0:
            hipLeft = self.lmlist[self.p1][:2]
            kneeLeft = self.lmlist[self.p2][:2]
            ankleLeft = self.lmlist[self.p3][:2]
            hipRight = self.lmlist[self.p4][:2]
            kneeRight = self.lmlist[self.p5][:2]
            ankleRight = self.lmlist[self.p6][:2]
            leftShoulder = self.lmlist[self.p7][:2]
            rightShoulder = self.lmlist[self.p8][:2]

            if len(hipLeft) >= 2 and len(kneeLeft) >= 2 and len(ankleLeft) >= 2 and len(hipRight) >= 2 and len(kneeRight) >= 2 and len(
                    ankleRight) >= 2 and len(leftShoulder) >= 2 and len(rightShoulder) >= 2:
                x1, y1 = hipLeft[:2]
                x2, y2 = kneeLeft[:2]
                x3, y3 = ankleLeft[:2]
                x4, y4 = hipRight[:2]
                x5, y5 = kneeRight[:2]
                x6, y6 = ankleRight[:2]
                x7, y7 = leftShoulder[:2]
                x8, y8 = rightShoulder[:2]

                vertical_line_angle = 90  # Góc của đường thẳng đứng so với trục x

                # calculating angle for left and right hands
                leftHandAngle = math.degrees(math.atan2(y1 - y2, x1 - x2) - math.atan2(y1 - y2, 0))
                rightHandAngle = math.degrees(math.atan2(y4 - y5, x4 - x5) - math.atan2(y4 - y5, 0))

                leftBackAngle = math.degrees(math.atan2(y1 - y2, 0) - math.atan2(y7 - y1, x7 - x1))
                rightBackAngle = math.degrees(math.atan2(y4 - y5, 0) - math.atan2(y8 - y4, x8 - x4))

                p2_p3_angle = math.degrees(math.atan2(y3 - y2, x3 - x2))
                p5_p6_angle = math.degrees(math.atan2(y6 - y5, x6 - x5))
                # Tính góc giữa đường thẳng đứng và đường thẳng nối giữa đầu gối và mắt cá chân
                leftkneeAngleLineAngle = abs(vertical_line_angle - p2_p3_angle)
                rightkneeAngleLineAngle = abs(vertical_line_angle - p5_p6_angle)

                # print (f"leftHandAngle: {leftHandAngle}")
                # print (f"rightHandAngle: {rightHandAngle}")
                # print (f"leftBackAngle: {leftBackAngle}")
                # print (f"rightBackAngle: {rightBackAngle}")
                # print (f"leftkneeAngleLineAngle: {leftkneeAngleLineAngle}")
                # print (f"rightkneeAngleLineAngle: {rightkneeAngleLineAngle}")
                

                leftHandAngle = int(np.interp(leftHandAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3
                rightHandAngle = int(np.interp(rightHandAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3
                leftBackAngle = int(np.interp(leftBackAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3
                rightBackAngle = int(np.interp(rightBackAngle, [0, 95], [0, 100])) # Ánh xạ sang từ 0-95 về 0-100 vì góc càng nhỏ thì càng tiến về trạng thái s3

                # drawing circles and lines on selected points
                if self.drawPoints:
                    cv2.circle(img, (x1, y1), 2, (0, 255, 0), 6)          
                    cv2.circle(img, (x2, y2), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x3, y3), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x4, y4), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x5, y5), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x6, y6), 2, (0, 255, 0), 6)            
                    cv2.circle(img, (x7, y7), 2, (0, 255, 0), 6)           
                    cv2.circle(img, (x8, y8), 2, (0, 255, 0), 6)

                    cv2.line(img, (x1, y1), (x2, y2), (0, 0, 255), 2)
                    cv2.line(img, (x2, y2), (x3, y3), (0, 0, 255), 2)
                    cv2.line(img, (x4, y4), (x5, y5), (0, 0, 255), 2)
                    cv2.line(img, (x5, y5), (x6, y6), (0, 0, 255), 2)
                    cv2.line(img, (x1, y1), (x4, y4), (0, 0, 255), 2)
                    cv2.line(img, (x1, y1), (x7, y7), (0, 0, 255), 2)
                    cv2.line(img, (x4, y4), (x8, y8), (0, 0, 255), 2)

                return [leftHandAngle, rightHandAngle, leftBackAngle, rightBackAngle, leftkneeAngleLineAngle, rightkneeAngleLineAngle]

# defining some variables
counter = 0
direction = 0
error1 = False
error2 = False

error1_start_time = None
error2_start_time = None

PREDICTION_PROB_THRESHOLD = 0.7

# Error vars
VISIBILITY_THRESHOLD = 0.6
FOOT_SHOULDER_RATIO_THRESHOLDS = [1.2, 2.8]
KNEE_FOOT_RATIO_THRESHOLDS = {
    "up": [0.5, 1.0],
    "middle": [0.7, 1.0],
    "down": [0.7, 1.1],
}

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while True:
        ret, img = cap.read()
        # img = cv2.resize(img, (640, 480))
        img = rescale_frame(img, percent=40)

        if not ret:
            print("Ignoring empty camera frame.")
            break
                
        # resize frame để tăng tốc độ xử lý
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = pose.process(img)

        if not results.pose_landmarks:
            print("No human found")
            continue

        # Cần khôi phục lại màu gốc của ảnh
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

        key_points = extract_amd_recalculate_landmarks(results.pose_landmarks.landmark)
        X = pd.DataFrame([key_points], columns=HEADERS[1:])
        X = input_scaler.transform(X)

        predicted_class = model.predict(X)[0]
        predicted_class = get_class(model.predict(X)[0])

        # putting predicted class on the screen
        cv2.putText(img, f"State: {predicted_class}", (10, 420), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)


        detector.findPose(img, draw=0)
        lmList, bboxInfo = detector.findPosition(img, bboxWithHands=0, draw=False)
        
        angle1 = angleFinder(lmList, 23, 25, 27, 24, 26, 28, 11, 12, drawPoints=True)
        hands = angle1.angle()
        left, right, leftBackAngle, rightBackAngle, leftkneeAngleLineAngle, rightkneeAngleLineAngle = hands[0:]

        # Set lại giá trị ban đầu
        error1 = False
        error2 = False

        # Counting number of squat
        if left > 75 and right > 75:
            if direction == 0:
                counter += 0.5
                direction = 1
        if left <= 70 and right <= 70:
            if direction == 1:
                counter += 0.5
                direction = 0

        # Analyze squat pose
            analyzed_results = analyze_foot_knee_placement(results=results, stage=predicted_class, foot_shoulder_ratio_thresholds=FOOT_SHOULDER_RATIO_THRESHOLDS, knee_foot_ratio_thresholds=KNEE_FOOT_RATIO_THRESHOLDS, visibility_threshold=VISIBILITY_THRESHOLD)

            foot_placement_evaluation = analyzed_results["foot_placement"]
            knee_placement_evaluation = analyzed_results["knee_placement"]

            print(f"Foot placement: {foot_placement_evaluation}")
            print(f"Knee placement: {knee_placement_evaluation}")

            current_time = time.time()
            
            # * Evaluate FOOT PLACEMENT error
            if foot_placement_evaluation == 1 or foot_placement_evaluation == 2:
                if foot_placement_evaluation == 1:
                    foot_placement = "Foot is too tight"
                elif foot_placement_evaluation == 2:
                    foot_placement = "Foot is too wide"
                error1 = True
                if error1_start_time is None:  # Nếu lỗi mới xuất hiện
                    error1_start_time = current_time
            elif error1_start_time is not None:
                error1_start_time = None  # Reset thời gian nếu lỗi không còn
            
            if knee_placement_evaluation == 1 or knee_placement_evaluation == 2:
                if knee_placement_evaluation == 1:
                    knee_placement = "Knee is too tight"
                elif knee_placement_evaluation == 2:
                    knee_placement = "Knee is too wide"
                error2 = True
                if error2_start_time is None:  # Nếu lỗi mới xuất hiện
                    error2_start_time = current_time
            elif error2_start_time is not None:
                error2_start_time = None  # Reset thời gian nếu lỗi không còn

        # putting scores on the screen
        cv2.rectangle(img, (0, 0), (120, 120), (255, 0, 0), -1)
        cv2.putText(img, str(int(counter)), (1, 70), cv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1.6, (0, 0, 255), 6)

        # Draw errors
        if error1 and error1_start_time  and (current_time - error1_start_time) < 1000:
            cv2.rectangle(img, (380, 10), (630, 50), (0, 215, 215), -1)
            cv2.putText(img, foot_placement, (390, 30), cv2.FONT_HERSHEY_TRIPLEX , 0.7, (59, 59, 56), 3)
        if error2 and error2_start_time and (current_time - error2_start_time) < 1000:
            cv2.rectangle(img, (380, 60), (630, 100), (0, 215, 215), -1)
            cv2.putText(img, knee_placement, (390, 80), cv2.FONT_HERSHEY_TRIPLEX , 0.7, (59, 59, 56), 3)

        # Converting values for rectangles
        leftval = np.interp(left, [0, 100], [400, 200])
        rightval = np.interp(right, [0, 100], [400, 200])

        # For color changing
        value_left = np.interp(left, [0, 100], [0, 100])
        value_right = np.interp(right, [0, 100], [0, 100])

        # Drawing right rectangle and putting text
        cv2.putText(img, 'R', (24, 195), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 5)
        cv2.rectangle(img, (8, 200), (50, 400), (0, 255, 0), 5)
        cv2.rectangle(img, (8, int(rightval)), (50, 400), (255, 0, 0), -1)

        # Drawing right rectangle and putting text
        cv2.putText(img, 'L', (604, 195), cv2.FONT_HERSHEY_DUPLEX, 1, (255, 0, 0), 5)
        cv2.rectangle(img, (582, 200), (632, 400), (0, 255, 0), 5)
        cv2.rectangle(img, (582, int(leftval)), (632, 400), (255, 0, 0), -1)

        # Tô màu đỏ khi góc đạt đến trạng thái s3
        if value_left > 75:
            cv2.rectangle(img, (582, int(leftval)), (632, 400), (0, 0, 255), -1)

        if value_right > 75:
            cv2.rectangle(img, (8, int(rightval)), (50, 400), (0, 0, 255), -1)

        cv2.imshow("Image", img)
        cv2.waitKey(1)

        # Nhấn q để thoát
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 1
Foot placement: 0
Knee placement: 1
Foot placement: 0
Knee placement: 1
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee placement: 0
Foot placement: 0
Knee place

AttributeError: 'NoneType' object has no attribute 'shape'